<a href="https://colab.research.google.com/github/jboca-andes/Recursos/blob/main/Semana_6_Construir_una_ETL_con_historia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar xf spark-3.2.0-bin-hadoop2.7.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"

In [3]:
!pip install -q findspark
!pip install -q pyspark
!pip install -q spark-df-profiling

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 46.2 MB/s 
     |████████████████████████████████| 91 kB 4.3 MB/s 


In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
aeropuertos = 'aeropuertos_cambios_infraestructura.csv'
vuelos = 'vuelosEtapa2.csv'

from pyspark import SparkFiles
spark.sparkContext.addFile(f'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%202/{aeropuertos}')
spark.sparkContext.addFile(f'https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%202/{vuelos}')

#Adicionando la descarga de cobertura
!wget https://github.com/MIAD-Modelo-Datos/Recursos/raw/main/Infraestructura%20visible/Etapa%202/Cobertura%20Aerea%20de%20Centros%20poblados%20por%20categoria%20de%20AereopuertoConCodigoCentroPoblado.xlsx

import pandas as pd
pd_cobertura = pd.read_excel("Cobertura Aerea de Centros poblados por categoria de AereopuertoConCodigoCentroPoblado.xlsx")

--2021-11-21 14:16:46--  https://github.com/MIAD-Modelo-Datos/Recursos/raw/main/Infraestructura%20visible/Etapa%202/Cobertura%20Aerea%20de%20Centros%20poblados%20por%20categoria%20de%20AereopuertoConCodigoCentroPoblado.xlsx
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%202/Cobertura%20Aerea%20de%20Centros%20poblados%20por%20categoria%20de%20AereopuertoConCodigoCentroPoblado.xlsx [following]
--2021-11-21 14:16:46--  https://raw.githubusercontent.com/MIAD-Modelo-Datos/Recursos/main/Infraestructura%20visible/Etapa%202/Cobertura%20Aerea%20de%20Centros%20poblados%20por%20categoria%20de%20AereopuertoConCodigoCentroPoblado.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw

In [7]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

#Estructura para la cobertura (Todo en String para no perder información)
esquema = StructType([StructField("codCentroPoblado", StringType(), True),
                      StructField("NomCentroPoblado", StringType(), True),
                      StructField("Aeropuerto", StringType(), True),
                      StructField("Distancia", DoubleType(), True),
                      StructField("Cobertura", StringType(), True),
                      StructField("Aerodromo", StringType(), True),
                      StructField("D_Aerodromo", DoubleType(), True),
                      StructField("Regional", StringType(), True),
                      StructField("D_Regional", DoubleType(), True),
                      StructField("Nacional", StringType(), True),
                      StructField("D_Nacional", DoubleType(), True),
                      StructField("Internacional", StringType(), True),
                      StructField("D_Internacional", DoubleType(), True),
                      StructField("Tipo_Cobertura", StringType(), True)
                      ])

In [8]:
df_aeropuertos = spark.read.csv('file://'+SparkFiles.get(aeropuertos), header=True, inferSchema=True)
df_vuelos = spark.read.csv('file://'+SparkFiles.get(vuelos), header=True, inferSchema=True)
df_cobertura = spark.createDataFrame(pd_cobertura, schema=esquema)

df_aeropuertos.createOrReplaceTempView("aeropuertos")
df_vuelos.createOrReplaceTempView("vuelos")
df_cobertura.createOrReplaceTempView("cobertura")

In [9]:
spark.sql('''
    select ROW_NUMBER() OVER (PARTITION BY sigla ORDER BY cast(UNIX_TIMESTAMP(concat('01','/','01','/',cast(ano as string)), 'MM/dd/yyyy') as timestamp)) posicion,
    sigla, iata, nombre, municipio, departamento, 
    categoria, cast(latitud as string) latitud, cast(longitud as string) longitud, 
    propietario, explotador, longitud_pista, ancho_pista, cast(pbmo as double) pbmo, 
    elevacion, resolucion, cast(fecha_construccion as date) fecha_construccion,
    cast(fecha_vigencia as date) fecha_vigencia, clase, tipo, cast(numero_vuelos_origen as int) numero_vuelos_origen,
    cast(gcd_departamento as string) gcd_departamento, cast(gcd_municipio as string) gcd_municipio,
    cast(UNIX_TIMESTAMP(concat('01','/','01','/',cast(ano as string)), 'MM/dd/yyyy') as timestamp) fechaInicioReg
    from aeropuertos
''').createOrReplaceTempView("aeropuertos_ok")


spark.sql('''
  select cast(ano as string) ano, cast(mes as string) mes, origen, destino, tipo_equipo, tipo_vuelo, trafico, empresa, 
  cast(vuelos as double) vuelos, cast(sillas as double) sillas, carga_ofrecida, cast(pasajeros as double) pasajeros, carga_bordo,
  cast(UNIX_TIMESTAMP(concat(case length(mes) when 1 then '0' else '' end, cast(mes as string),'/','01','/',cast(ano as string)), 'MM/dd/yyyy') as timestamp) fecha
  from vuelos where mes>=0 and ano >=0 and origen <> destino
''').createOrReplaceTempView('vuelos_ok')


spark.sql('''
  select *
  from cobertura
''').createOrReplaceTempView('cobertura_ok')

In [10]:
#Creacion de tablas
#Dimension de tiempo
tabla = "DIM_TIEMPO"
query = "SELECT ROW_NUMBER() OVER (ORDER BY FECHA) SRK_FECHA, * FROM (SELECT DISTINCT fecha FECHA, YEAR(fecha) ANHO, ceil(month(fecha)/6) SEMESTRE,  ceil(month(fecha)/3) TRIMESTRE, MONTH(fecha) MES, DAY(fecha) DIA from vuelos_ok)"
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [11]:
spark.sql("select * from DIM_TIEMPO").show(10)

+---------+-------------------+----+--------+---------+---+---+
|SRK_FECHA|              FECHA|ANHO|SEMESTRE|TRIMESTRE|MES|DIA|
+---------+-------------------+----+--------+---------+---+---+
|        1|2010-01-01 00:00:00|2010|       1|        1|  1|  1|
|        2|2010-02-01 00:00:00|2010|       1|        1|  2|  1|
|        3|2010-03-01 00:00:00|2010|       1|        1|  3|  1|
|        4|2010-04-01 00:00:00|2010|       1|        2|  4|  1|
|        5|2010-05-01 00:00:00|2010|       1|        2|  5|  1|
|        6|2010-06-01 00:00:00|2010|       1|        2|  6|  1|
|        7|2010-07-01 00:00:00|2010|       2|        3|  7|  1|
|        8|2010-08-01 00:00:00|2010|       2|        3|  8|  1|
|        9|2010-09-01 00:00:00|2010|       2|        3|  9|  1|
|       10|2010-10-01 00:00:00|2010|       2|        4| 10|  1|
+---------+-------------------+----+--------+---------+---+---+
only showing top 10 rows



In [12]:
#Creacion de tablas
#Dimension de Clase
tabla = "DIM_CLASE"
query_base = 'select DISTINCT tipo_equipo TIPO_EQUIPO, tipo_vuelo TIPO_VUELO, trafico TRAFICO, empresa EMPRESA from vuelos_ok'
query = "SELECT ROW_NUMBER() OVER (ORDER BY TIPO_EQUIPO, TIPO_VUELO, TRAFICO, EMPRESA) SRK_CLASE, * FROM ({})".format(query_base)
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [13]:
spark.sql("select * from DIM_CLASE").show(10)

+---------+-----------+----------+-------+----------------+
|SRK_CLASE|TIPO_EQUIPO|TIPO_VUELO|TRAFICO|         EMPRESA|
+---------+-----------+----------+-------+----------------+
|        1|        318|         A|      N|         AVIANCA|
|        2|        318|         C|      N|         AVIANCA|
|        3|        318|         R|      N|         AVIANCA|
|        4|        319|         A|      N|         AVIANCA|
|        5|        319|         C|      N|         AVIANCA|
|        6|        319|         R|      N|         AVIANCA|
|        7|        319|         R|      N|             SAM|
|        8|        330|         A|      N|         AVIANCA|
|        9|        330|         R|      N|         AVIANCA|
|       10|        332|         R|      N|TAMPA CARGO S.A.|
+---------+-----------+----------+-------+----------------+
only showing top 10 rows



In [14]:
#Creacion de tablas
#Dimension de Aeropuertos
tabla = "DIM_AEROPUERTO"

query_base = '''select DISTINCT a1.iata IATA, a1.sigla SIGLA, a1.nombre NOMBRE
, a1.municipio MUNICIPIO, 
a1.departamento DEPARTAMENTO, a1.categoria CATEGORIA, a1.latitud LATITUD,
a1.longitud LONGITUD,
a1.propietario PROPIETARIO, a1.explotador EXPLOTADOR, 
a1.longitud_pista LONGITUD_PISTA, a1.ancho_pista ANCHO_PISTA, 
a1.pbmo PBMO, a1.elevacion ELEVACION, a1.clase CLASE, a1.tipo TIPO,
CASE WHEN a1.posicion=1 THEN a1.fecha_construccion ELSE a1.fechaInicioReg END FECHA_INICIO, a2.fechaInicioReg FECHA_FINALIZACION
FROM aeropuertos_ok a1 LEFT JOIN aeropuertos_ok a2 ON a1.sigla=a2.sigla AND a1.posicion = a2.posicion-1 '''

query = "SELECT ROW_NUMBER() OVER (ORDER BY SIGLA, FECHA_INICIO) SRK_AEROPUERTO, * FROM ({})".format(query_base)
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [15]:
spark.sql("select * from DIM_AEROPUERTO").show(10)

+--------------+----+-----+--------------------+-------------+------------+---------+-------+--------+--------------------+-------------------+--------------+-----------+------+---------+-----+----------+-------------------+-------------------+
|SRK_AEROPUERTO|IATA|SIGLA|              NOMBRE|    MUNICIPIO|DEPARTAMENTO|CATEGORIA|LATITUD|LONGITUD|         PROPIETARIO|         EXPLOTADOR|LONGITUD_PISTA|ANCHO_PISTA|  PBMO|ELEVACION|CLASE|      TIPO|       FECHA_INICIO| FECHA_FINALIZACION|
+--------------+----+-----+--------------------+-------------+------------+---------+-------+--------+--------------------+-------------------+--------------+-----------+------+---------+-----+----------+-------------------+-------------------+
|             1|null|  7FO|             LA ISLA|Puerto Gaitán|        Meta|Aeródromo| 4.4211|-71.6271|LA ISLA Y EL ROSA...|      LA CEIBA S.A.|        1079.0|       19.0|3000.0|    538.0|   1A|Fumigación|2015-06-05 00:00:00|2016-01-01 00:00:00|
|             2|null

In [16]:
tabla = 'DIM_CENTRO_POBLADO'
query_base = '''
SELECT DISTINCT codCentroPoblado COD_CENTRO_POBLADO, NomCentroPoblado NOMBRE_CENTRO_POBLADO, Tipo_Cobertura TIPO_COBERTURA FROM cobertura_ok
'''
query = "SELECT ROW_NUMBER() OVER (ORDER BY COD_CENTRO_POBLADO) SRK_CENTRO_POBLADO, * FROM ({})".format(query_base)
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [17]:
spark.sql("select * from DIM_CENTRO_POBLADO").show(10)

+------------------+------------------+---------------------+--------------+
|SRK_CENTRO_POBLADO|COD_CENTRO_POBLADO|NOMBRE_CENTRO_POBLADO|TIPO_COBERTURA|
+------------------+------------------+---------------------+--------------+
|                 1|          11001000| BOGOTÁ, DISTRITO ...| Internacional|
|                 2|          11001002|             NAZARETH|     Aeródromo|
|                 3|          11001007|            PASQUILLA| Internacional|
|                 4|          11001008|             SAN JUAN|     Aeródromo|
|                 5|          11001009|              BETANIA|     Aeródromo|
|                 6|          11001010|             LA UNIÓN| Sin cobertura|
|                 7|          11001011|        MOCHUELO ALTO| Internacional|
|                 8|          11001012|           CHORRILLOS| Internacional|
|                 9|          11001013|           EL DESTINO| Internacional|
|                10|          11001014|        NUEVA GRANADA| Sin cobertura|

In [18]:
tabla = 'CENTRO_POBLADO_AEROPUERTO'
query_base = '''
SELECT CP.SRK_CENTRO_POBLADO, SRK_AEROPUERTO, C.Distancia DISTANCIA
FROM cobertura_ok C 
  INNER JOIN DIM_CENTRO_POBLADO CP ON C.codCentroPoblado = CP.COD_CENTRO_POBLADO
  INNER JOIN DIM_AEROPUERTO A1 ON C.Aeropuerto = A1.SIGLA AND A1.FECHA_FINALIZACION IS NULL
UNION
SELECT CP.SRK_CENTRO_POBLADO, SRK_AEROPUERTO, C.D_Aerodromo DISTANCIA
FROM cobertura_ok C 
  INNER JOIN DIM_CENTRO_POBLADO CP ON C.codCentroPoblado = CP.COD_CENTRO_POBLADO
  INNER JOIN DIM_AEROPUERTO A1 ON C.Aerodromo = A1.SIGLA AND A1.FECHA_FINALIZACION IS NULL
UNION
SELECT CP.SRK_CENTRO_POBLADO, SRK_AEROPUERTO, C.D_Regional DISTANCIA
FROM cobertura_ok C 
  INNER JOIN DIM_CENTRO_POBLADO CP ON C.codCentroPoblado = CP.COD_CENTRO_POBLADO
  INNER JOIN DIM_AEROPUERTO A1 ON C.Regional = A1.SIGLA AND FECHA_FINALIZACION IS NULL
UNION
SELECT CP.SRK_CENTRO_POBLADO, SRK_AEROPUERTO, C.D_Nacional DISTANCIA
FROM cobertura_ok C 
  INNER JOIN DIM_CENTRO_POBLADO CP ON C.codCentroPoblado = CP.COD_CENTRO_POBLADO
  INNER JOIN DIM_AEROPUERTO A1 ON C.Nacional = A1.SIGLA AND FECHA_FINALIZACION IS NULL
UNION
SELECT CP.SRK_CENTRO_POBLADO, SRK_AEROPUERTO, C.D_Internacional DISTANCIA
FROM cobertura_ok C 
  INNER JOIN DIM_CENTRO_POBLADO CP ON C.codCentroPoblado = CP.COD_CENTRO_POBLADO
  INNER JOIN DIM_AEROPUERTO A1 ON C.Internacional = A1.SIGLA AND FECHA_FINALIZACION IS NULL
'''

query = "SELECT ROW_NUMBER() OVER (ORDER BY SRK_CENTRO_POBLADO) SRK_CENTRO_POBLADO_AEROPUERTO, * FROM ({})".format(query_base)
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [19]:
spark.sql("select * from CENTRO_POBLADO_AEROPUERTO").show(10)

+-----------------------------+------------------+--------------+----------------+
|SRK_CENTRO_POBLADO_AEROPUERTO|SRK_CENTRO_POBLADO|SRK_AEROPUERTO|       DISTANCIA|
+-----------------------------+------------------+--------------+----------------+
|                            1|                 1|           659|7.21322529297458|
|                            2|                 1|           716| 18.647530806448|
|                            3|                 1|           845|76.5051883878193|
|                            4|                 2|           389|28.6288894672103|
|                            5|                 2|           845|59.0531776161181|
|                            6|                 2|           659| 58.844770553807|
|                            7|                 3|           389|20.4466959254748|
|                            8|                 3|           845|67.4331859343624|
|                            9|                 3|           659|28.6349893826413|
|   

In [20]:
#Creacion de tablas
#Tabla de hechos de vuelos
tabla = "FACT_VUELO"
query_base = '''
SELECT 'SALIENTE' TIPO, 
A.SRK_AEROPUERTO, 
F.SRK_FECHA,
C.SRK_CLASE,
vuelos VUELOS,
sillas SILLAS,
carga_ofrecida CARGA_OFRECIDA,
pasajeros PASAJEROS,
carga_bordo CARGA_BORDO
FROM vuelos_ok V INNER JOIN DIM_AEROPUERTO A on V.origen = a.SIGLA and V.fecha >= A.FECHA_INICIO AND V.fecha < IFNULL(A.FECHA_FINALIZACION, current_date())
INNER JOIN DIM_TIEMPO F ON V.fecha = F.FECHA
INNER JOIN DIM_CLASE C ON V.tipo_equipo = C.TIPO_EQUIPO AND V.tipo_vuelo = C.TIPO_VUELO AND V.trafico = C.TRAFICO AND V.empresa = C.EMPRESA
UNION
SELECT 'ENTRANTE' TIPO, 
A.SRK_AEROPUERTO, 
F.SRK_FECHA,
C.SRK_CLASE,
vuelos VUELOS,
sillas SILLAS,
carga_ofrecida CARGA_OFRECIDA,
pasajeros PASAJEROS,
carga_bordo CARGA_BORDO
FROM vuelos_ok V INNER JOIN DIM_AEROPUERTO A on V.destino = a.SIGLA and V.fecha >= A.FECHA_INICIO AND V.fecha < IFNULL(A.FECHA_FINALIZACION, current_date())
INNER JOIN DIM_TIEMPO F ON V.fecha = F.FECHA
INNER JOIN DIM_CLASE C ON V.tipo_equipo = C.TIPO_EQUIPO AND V.tipo_vuelo = C.TIPO_VUELO AND V.trafico = C.TRAFICO AND V.empresa = C.EMPRESA
'''
query = "SELECT ROW_NUMBER() OVER (ORDER BY TIPO, SRK_AEROPUERTO, SRK_FECHA, SRK_CLASE) SRK_FACT_VUELO, * FROM ({})".format(query_base)
crear_tabla = "CREATE TABLE {} USING CSV AS {}".format(tabla, query)
spark.sql(crear_tabla)

DataFrame[]

In [21]:
spark.sql("select * from FACT_VUELO").show(10)

+--------------+--------+--------------+---------+---------+------+------+--------------+---------+-----------+
|SRK_FACT_VUELO|    TIPO|SRK_AEROPUERTO|SRK_FECHA|SRK_CLASE|VUELOS|SILLAS|CARGA_OFRECIDA|PASAJEROS|CARGA_BORDO|
+--------------+--------+--------------+---------+---------+------+------+--------------+---------+-----------+
|             1|ENTRANTE|             3|       41|      163|   1.0|   0.0|           0.0|      3.0|       30.0|
|             2|ENTRANTE|             3|       41|      328|   2.0|   0.0|           0.0|      2.0|       50.0|
|             3|ENTRANTE|             3|       43|      169|   1.0|   0.0|           0.0|      1.0|      100.0|
|             4|ENTRANTE|             3|       43|      354|   1.0|   0.0|           0.0|      4.0|        0.0|
|             5|ENTRANTE|             3|       44|      163|   1.0|   0.0|           0.0|      1.0|       90.0|
|             6|ENTRANTE|             3|       45|      169|   1.0|   0.0|           0.0|      3.0|     

In [22]:
spark.sql('DESCRIBE TABLE DIM_TIEMPO').show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|SRK_FECHA|      int|   null|
|    FECHA|timestamp|   null|
|     ANHO|      int|   null|
| SEMESTRE|   bigint|   null|
|TRIMESTRE|   bigint|   null|
|      MES|      int|   null|
|      DIA|      int|   null|
+---------+---------+-------+



In [23]:
spark.sql('DESCRIBE TABLE DIM_CLASE').show()

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|  SRK_CLASE|      int|   null|
|TIPO_EQUIPO|   string|   null|
| TIPO_VUELO|   string|   null|
|    TRAFICO|   string|   null|
|    EMPRESA|   string|   null|
+-----------+---------+-------+



In [24]:
spark.sql('DESCRIBE TABLE DIM_AEROPUERTO').show()

+------------------+---------+-------+
|          col_name|data_type|comment|
+------------------+---------+-------+
|    SRK_AEROPUERTO|      int|   null|
|              IATA|   string|   null|
|             SIGLA|   string|   null|
|            NOMBRE|   string|   null|
|         MUNICIPIO|   string|   null|
|      DEPARTAMENTO|   string|   null|
|         CATEGORIA|   string|   null|
|           LATITUD|   string|   null|
|          LONGITUD|   string|   null|
|       PROPIETARIO|   string|   null|
|        EXPLOTADOR|   string|   null|
|    LONGITUD_PISTA|   double|   null|
|       ANCHO_PISTA|   double|   null|
|              PBMO|   double|   null|
|         ELEVACION|   double|   null|
|             CLASE|   string|   null|
|              TIPO|   string|   null|
|      FECHA_INICIO|timestamp|   null|
|FECHA_FINALIZACION|timestamp|   null|
+------------------+---------+-------+



In [25]:
spark.sql('DESCRIBE TABLE DIM_CENTRO_POBLADO').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|  SRK_CENTRO_POBLADO|      int|   null|
|  COD_CENTRO_POBLADO|   string|   null|
|NOMBRE_CENTRO_POB...|   string|   null|
|      TIPO_COBERTURA|   string|   null|
+--------------------+---------+-------+



In [26]:
spark.sql('DESCRIBE TABLE CENTRO_POBLADO_AEROPUERTO').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|SRK_CENTRO_POBLAD...|      int|   null|
|  SRK_CENTRO_POBLADO|      int|   null|
|      SRK_AEROPUERTO|      int|   null|
|           DISTANCIA|   double|   null|
+--------------------+---------+-------+



In [27]:
spark.sql('DESCRIBE TABLE FACT_VUELO').show()

+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|SRK_FACT_VUELO|      int|   null|
|          TIPO|   string|   null|
|SRK_AEROPUERTO|      int|   null|
|     SRK_FECHA|      int|   null|
|     SRK_CLASE|      int|   null|
|        VUELOS|   double|   null|
|        SILLAS|   double|   null|
|CARGA_OFRECIDA|   double|   null|
|     PASAJEROS|   double|   null|
|   CARGA_BORDO|   double|   null|
+--------------+---------+-------+



In [ ]:
dfs = ['DIM_TIEMPO','DIM_AEROPUERTO','DIM_CLASE','DIM_CENTRO_POBLADO','FACT_VUELO','CENTRO_POBLADO_AEROPUERTO']
for d in dfs:
  dft = spark.table(d) 
  #dft.coalesce(1).write.csv(f"{d}.CSV", header=True)
  dft.write \
    .format("jdbc") \
    .option("url", "jdbc:sqlserver:dbserver") \
    .option(d, d) \
    .option("user", "username") \
    .option("password", "password") \
    .save()